In [1]:
import tensorflow as tf
import numpy as np
from random import seed
from random import random
from math import exp


# Initialization 
This is where we initialize the dataset and the hidden network. 

In [2]:

#Load Dataset
inputFilenameWithPath = 'train_data.txt'
inputData = np.loadtxt(inputFilenameWithPath, delimiter=",")
n_inputs = len(inputData[0]) - 1
n_outputs = len(set([row[-1] for row in inputData]))
# Initialize a single layer neural network with n_neurons in the hidden layer
def initialize_network(n_inputs, n_neurons , n_outputs):
	network = list()
	hidden_layer = [{'weights' :[random() for layer in range(n_inputs + 1)]} for i in range(n_neurons)]
	network.append(hidden_layer)
	output_layer = [{'weights' :[random() for layer in range(n_neurons + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

seed(1)
network = initialize_network(n_inputs, 3, n_outputs)


# Validate the input and weights initialization
Print the layer weights

In [3]:
#print(network)
for layer in network:
	print(layer)
    #print(len(layer))

#print(inputData)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}, {'weights': [0.2550690257394217, 0.49543508709194095, 0.4494910647887381]}, {'weights': [0.651592972722763, 0.7887233511355132, 0.0938595867742349]}]
[{'weights': [0.02834747652200631, 0.8357651039198697, 0.43276706790505337, 0.762280082457942]}, {'weights': [0.0021060533511106927, 0.4453871940548014, 0.7215400323407826, 0.22876222127045265]}]


# The product summation and sigmoid activation

In [4]:
# Calculate neuron activation for an input
def weights_input_product(weights, inputs):
	summation = weights[-1] # Add the bias first 
	for i in range(len(weights)-1):
		summation += weights[i] * inputs[i] #Add the product of the input and the weights
	return summation

In [5]:
def sigmoid(z):
	return 1.0 / (1.0 + exp(-z))

# Forward propagate

In [6]:
#Send the list of outputs for each layer
def forward_propagate(network, inputData):
    outputs = []
    inputRecord = inputData
    for layer in network: # Iterate over the layers
        layer_output = []
        for neuron in layer: # Iterate for all neurons
            summation = weights_input_product(neuron['weights'],inputRecord)
            activation = sigmoid(summation)
            neuron['output'] = activation
            layer_output.append(activation)
        outputs.append(layer_output)
        inputRecord = layer_output
    return layer_output
            
        

# Test the forward propagation

In [7]:
row = inputData[2]
output = forward_propagate(network, row)
print(output)

[0.8452790306067982, 0.7406450596041884]


In [8]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

In [9]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [10]:
# test backpropagation of error
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
#for layer in network:
	#print(layer)

In [11]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

In [12]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		#print(train)
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			#print("Expected shape",expected)
			expected[int(row[-1])] = 1
			#expected[1] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [13]:
seed(1)
#inputData = np.loadtxt('/Users/saur6410/Google Drive/VT/Dissertation/Deep Learning/Datasets/iris_numeric.csv', delimiter=",")
inputFilenameWithPath = 'train_data.txt'
inputData = np.loadtxt(inputFilenameWithPath, delimiter=",")
n_inputs = len(inputData[0]) - 1
n_outputs = len(set([row[-1] for row in inputData]))
#print(n_inputs)
#print(n_outputs)
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, inputData, 0.5, 20, n_outputs)
#for layer in network:
	#print(layer)

>epoch=0, lrate=0.500, error=71.736
>epoch=1, lrate=0.500, error=6.658
>epoch=2, lrate=0.500, error=4.771
>epoch=3, lrate=0.500, error=4.057
>epoch=4, lrate=0.500, error=3.677
>epoch=5, lrate=0.500, error=3.440
>epoch=6, lrate=0.500, error=3.276
>epoch=7, lrate=0.500, error=3.155
>epoch=8, lrate=0.500, error=3.062
>epoch=9, lrate=0.500, error=2.987
>epoch=10, lrate=0.500, error=2.926
>epoch=11, lrate=0.500, error=2.874
>epoch=12, lrate=0.500, error=2.829
>epoch=13, lrate=0.500, error=2.791
>epoch=14, lrate=0.500, error=2.756
>epoch=15, lrate=0.500, error=2.726
>epoch=16, lrate=0.500, error=2.698
>epoch=17, lrate=0.500, error=2.673
>epoch=18, lrate=0.500, error=2.650
>epoch=19, lrate=0.500, error=2.628


In [14]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

In [15]:
# Test making predictions with the network
inputFilenameWithPath = 'test_data.txt'
dataset = np.loadtxt(inputFilenameWithPath, delimiter=",")
for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0

Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1

Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
